# memory update details

see [studio/directive_memory_bot.py](studio/directive_memory_bot.py)

- Hey, how are you? I’m Evgeny, I’m a software engineer and I need your help with my project. This is a TaskManager Java Spring Boot application. I do not get how to configure security there! Also, I often come back to these answers later, so please include comments inside any code you share.
- Sorry if I sound a bit stressed — I’m not very experienced with Spring Security. If you could break things down step by step, that would really help.

In [1]:
from pydantic import BaseModel, Field

class InteractionInstruction(BaseModel):
    instruction: str = Field(
        description=(
            "An independent, atomic instruction that describes one specific way the assistant should adapt its interaction to the user. "
            "Each instruction should focus on a single behavior or style — do not combine multiple preferences into one. "
            "Instructions must be phrased as imperatives. "
            "Examples: "
            "'Explain technical topics using a step-by-step format.', "
            "'Use concise code examples.', "
            "'Avoid adding explanations unless explicitly requested.', "
            "'Include inline comments in code when requested.', "
            "'Ask if the user wants a deeper explanation after giving an answer.'"
        )
    )

class InteractionInstructionList(BaseModel):
    instructions: list[InteractionInstruction] = Field(
        description="A collection of interaction-relevant insights to guide how the assistant should respond to this user."
    )

In [2]:
from trustcall import create_extractor
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

model = ChatOpenAI(model="gpt-4o-mini")

trustcall_extractor = create_extractor(
    model,
    tools=[InteractionInstruction],
    tool_choice="InteractionInstruction",
    enable_inserts=True
)

result = trustcall_extractor.invoke([HumanMessage(content="""
Hey, how are you? I am Evgeny, I am a junior software engineer and I need your help with 
my project. This is a TaskManager Java Spring Boot application. I do not get how
to configure security there!
Also, I often come back to these answers later, so please include comments inside any code you share.
""")])

In [3]:
for m in result["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  InteractionInstruction (call_2c5YPrEj5qBkDve1WSZjpwMa)
 Call ID: call_2c5YPrEj5qBkDve1WSZjpwMa
  Args:
    instruction: Include inline comments in code when requested.


In [4]:
existing_instructions = [InteractionInstruction(instruction=result["responses"][0].instruction)]
existing_instructions

[InteractionInstruction(instruction='Include inline comments in code when requested.')]

## Update & Listen

In [5]:
class ToolsListener:
    def __init__(self):
        self.tools = []

    def __call__(self, execution):
        runs = [execution]
        while runs:
            run = runs.pop()
            if run.child_runs:
                runs.extend(run.child_runs)
            if run.run_type == "chat_model":
                self.tools.append(
                    run.outputs["generations"][0][0]["message"]["kwargs"]["tool_calls"]
                )

In [6]:
# prepare existing facts
schema = "InteractionInstruction"

memories = [(str(i + 1), schema, instruction.model_dump()) for i, instruction in enumerate(existing_instructions)]
memories

[('1',
  'InteractionInstruction',
  {'instruction': 'Include inline comments in code when requested.'})]

In [7]:
from langchain_core.messages import HumanMessage, AIMessage

listener = ToolsListener()
extractor_with_listener = trustcall_extractor.with_listeners(on_end=listener)


conversation = [
    HumanMessage(content="""
    Sorry if I sound a bit stressed — I’m not very experienced with Spring Security.
    If you could break things down step by step, that would really help.
    """),

    AIMessage(content="""
    Totally understood! No worries at all — I’ll walk you through it one step at a time.
    Let’s start with the basic configuration: you’ll need to add the Spring Security dependency first. Here's how...
    """), 

    HumanMessage(content="""
    Thanks, but could you skip the long explanation? Just tell me exactly what I need to do.
    """),
]

# Update the instruction
system_msg = """
Update existing directives about the user and create new ones based on the following conversation, 
ensuring each directive reflects one specific communication preference or behavior the assistant should follow.
"""


result = extractor_with_listener.invoke({
    "messages": [system_msg] + conversation, 
    "existing": memories
})

Could not apply patch: member '-' not found in {'instruction': 'Include inline comments in code when requested. Break things down step by step when the user expresses stress or lack of experience.'}


In [8]:
for m in result["response_metadata"]: 
    print(m)

{'id': 'call_HoDenLGYvhetL7PcbCcgxGVJ'}
{'id': 'call_PACSBOHxr1jcdRNO6TKYAEKY'}


In [9]:
for m in result["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  InteractionInstruction (call_HoDenLGYvhetL7PcbCcgxGVJ)
 Call ID: call_HoDenLGYvhetL7PcbCcgxGVJ
  Args:
    instruction: Break things down step by step when the user expresses stress or lack of experience.
  InteractionInstruction (call_PACSBOHxr1jcdRNO6TKYAEKY)
 Call ID: call_PACSBOHxr1jcdRNO6TKYAEKY
  Args:
    instruction: Skip long explanations and provide direct instructions when asked.


In [10]:
listener.tools

[[{'name': 'PatchDoc',
   'args': {'json_doc_id': '1',
    'planned_edits': 'Update the existing instruction to include breaking down information step by step. Add a new instruction for concise communication without lengthy explanations.',
    'patches': [{'op': 'replace',
      'path': '/instruction',
      'value': 'Include inline comments in code when requested. Break things down step by step when the user expresses stress or lack of experience.'},
     {'op': 'add',
      'path': '/-/',
      'value': 'Skip long explanations and provide direct instructions when asked.'}]},
   'id': 'call_VeGv0C3Dg5G8Q9kV65uyU26z',
   'type': 'tool_call'},
  {'name': 'InteractionInstruction',
   'args': {'instruction': 'Break things down step by step when the user expresses stress or lack of experience.'},
   'id': 'call_HoDenLGYvhetL7PcbCcgxGVJ',
   'type': 'tool_call'},
  {'name': 'InteractionInstruction',
   'args': {'instruction': 'Skip long explanations and provide direct instructions when aske

In [ ]:
def summarize_listener_tools(tools_data):
    if not tools_data:
        print("No tool calls recorded.")
        return

    for i, tool_calls in enumerate(tools_data):
        print(f"\n=== Tool Calls for Document {i} ===")
        for call in tool_calls:
            name = call.get("name")
            args = call.get("args", {})
            if name == "PatchDoc":
                doc_id = args.get("json_doc_id", "Unknown")
                plan = args.get("planned_edits", "No plan provided.")
                patches = args.get("patches", [])

                print(f"\n🛠️ Document {doc_id} updated:")
                print(f"📝 Plan: {plan}")
                for patch in patches:
                    op = patch.get("op")
                    path = patch.get("path")
                    value = patch.get("value")
                    print(f"🔧 Patch: {op.upper()} {path} → {value}")

            else:
                print(f"\n➕ New memory created:")
                print(f"📌 Tool: {name}")
                print(f"📄 Content: {args}")

In [ ]:
summarize_listener_tools(listener.tools)